
#**RELIABLE RAG IMPLEMENTATION**

## Downloading Neccesary libraries

In [ ]:
pip install python-dotenv

In [ ]:
pip install langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


In [ ]:
pip install langchain_cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.8 MB/s eta 0:00:00


In [ ]:
pip install langchain_groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.3 MB/s eta 0:00:00


In [ ]:
pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 28.9 MB/s eta 0:00:00


In [ ]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 33.5 MB/s eta 0:00:00


## Importing Neccesary Libraries

In [ ]:
import os
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"


In [ ]:
import os
from dotenv import load_dotenv

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
from enum import Enum
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.output_parsers import StrOutputParser
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

## Setting up API keys


In [ ]:
load_dotenv(load_dotenv(dotenv_path=".env"))
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
COHERE_API_KEY = os.getenv('COHERE_API_KEY')



## Embedding and indexing and retrieving the document


In [ ]:
embedding_model = CohereEmbeddings(model="embed-english-v3.0")



loader = PyMuPDFLoader(r"final_draft.pdf")
docs = loader.load()  # Returns LangChain documents with page_content and metadata
docs_list=docs

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)
doc_splits = [doc for doc in doc_splits if doc.page_content.strip()]

print(f"doc_splits count: {len(doc_splits)}")

try:
    print("Creating FAISS vectorstore...")
    vectorstore = FAISS.from_documents(doc_splits, embedding=embedding_model)
    print("FAISS vectorstore created.")
except Exception as e:
    print("Vectorstore creation failed:", e)
    exit()

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={'k': 4})


question = "What are RAGs"
retrieved_docs = retriever.invoke(question)




doc_splits count: 84
Creating FAISS vectorstore...
FAISS vectorstore created.


## Checking if a document is relevent or not and retaining only the relevant ones


In [ ]:
class BinaryScore(str, Enum):
    yes = "yes"
    no = "no"

class GradeDocuments(BaseModel):
    """Binary score: Is the document relevant to the user's question?"""
    binary_score: BinaryScore = Field(
        description="Answer 'yes' if the document is relevant to the question, else 'no'"
    )

llm = ChatGroq(model="llama3-8b-8192", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)


system_prompt = (
    """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
)
grade_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "Document: {document}\n\nQuestion: {question}")
])
retrieval_grader = grade_prompt | structured_llm_grader


docs_to_use = []

print("\n Scoring retrieved documents for relevance...\n")

for i, doc in enumerate(retrieved_docs):
    preview = doc.page_content[:400].strip()
    print(f"\nDoc {i + 1} Preview:\n{preview}\n{'-' * 60}")
    try:
        result = retrieval_grader.invoke({
            "document": doc.page_content[:500],
            "question": question
        })
        print(" Grader result:", result)

        if result.binary_score == BinaryScore.yes:
            docs_to_use.append(doc)

    except Exception as e:
        print(f" Grading failed for doc {i + 1}: {e}")

if not docs_to_use:
    print("\n No relevant documents retained after grading. Exiting early.")
    exit()

print(f"\n Relevant documents retained: {len(docs_to_use)}")






 Scoring retrieved documents for relevance...


Doc 1 Preview:
Chapter 7
Retrival Augumented
Generation:RAGs
Abstract
This is a chapter focuses on Retrieval-Augmented Generation a special technique developed to counter
the problem of hallucination in LLMs. This chapters provides an overview on Motivation, Working,
models, training, decoding and python implementation of RAGs.
7.1
Introduction
RAG stands for Retrieval-Augmented Generation.
RAG is a hybrid appro
------------------------------------------------------------
 Grader result: binary_score=<BinaryScore.yes: 'yes'>

Doc 2 Preview:
Chapter 10
ABSTRACT BASE CLASSES,
ABSTRACT FACTORY,
ADVANCED AND AGENTIC
RAGs
Abstract
This chapter throws light on some of the working components of Llama-index and also the advanced
RAG implementation techniques. It summarizes the Advanced RAG methods, Agentic RAG methods
and examples of orchestration methods.
10.1
Abstract Base Classes and Abstract Factory
LlamaIndex, like many extensible Pytho
---

## Setting up LLM to answer our question

In [ ]:
system = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge.
Use three-to-five sentences maximum and keep the answer concise."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system),
        ("human", "Retrived documents: \n\n <docs>{documents}</docs> \n\n User question: <question>{question}</question>")
    ]
)

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)

def format_docs(docs):
    return "\n".join(f"<doc{i+1}>:\nTitle:{doc.metadata['title']}\nSource:{doc.metadata['source']}\nContent:{doc.page_content}\n</doc{i+1}>\n" for i, doc in enumerate(docs))

rag_chain = prompt | llm | StrOutputParser()

generation = rag_chain.invoke({"documents":format_docs(docs_to_use), "question": question})
print(generation)
#print("\n Hello \n")



RAGs stands for Retrieval-Augmented Generation, a hybrid approach that combines pre-trained, parametric-memory generation models with a non-parametric memory (an external knowledge source) to improve the performance of large language models. This technique helps counter the problem of hallucination in LLMs by accessing and utilizing external knowledge.


# Sets up an agent which grades hallucination present in the answer.

In [ ]:
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generated answer"""
    binary_score: str = Field(
        ...,
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

llm = ChatGroq(model="llama3-8b-8192", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

system = """You are a grader evaluating whether an AI-generated answer is grounded in or supported by a set of retrieved documents.

Your task is to determine whether the content of the answer is factually supported by any part of the provided documents, even if the phrasing is different.

Please tolerate paraphrasing, synonym usage, and reworded content as long as the meaning remains consistent with the retrieved evidence.

Respond strictly with a binary score: 'yes' or 'no'.

- 'yes' → The answer is factually supported by the retrieved documents (even if rephrased).
- 'no' → The answer includes information not found in the documents or contradicts them."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n <facts>{documents}</facts> \n\n LLM generation: <generation>{generation}</generation>"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader

response = hallucination_grader.invoke({"documents": format_docs(docs_to_use), "generation": generation})
print(response)





binary_score='yes'


## Sets up an agent which finds the exact segments in our document from which the answer is generated from.

In [ ]:
class HighlightDocuments(BaseModel):
    """Return specific part of a document used for answering the question"""

    id: List[str] = Field(
        ...,
        description="List of id of docs used to answers the question"
    )

    title: List[str] = Field(
        ...,
        description="List of titles used to answers the question"
    )

    source: List[str] = Field(
        ...,
        description="List of sources used to answers the question"
    )

    segment: List[str] = Field(
        ...,
        description="List of direct segements from used documents that answers the question"
    )


llm = ChatGroq(model="llama3-70b-8192", temperature=0)

parser = PydanticOutputParser(pydantic_object=HighlightDocuments)


system = """You are a highly capable assistant for document attribution and traceability.

Your task is to analyze a set of documents that were used to generate an answer to a user's question, and extract **verbatim text segments** from those documents that directly match or support the content found in the answer.

You are provided with:
- A user's question.
- An answer that has already been generated.
- A set of documents that were used while generating that answer.

Your job is to carefully compare the answer with the documents, and extract **word-for-word matching text** from the documents that clearly contributed to the answer. These may be facts, phrases, or sentences from different documents.

Instructions:
1. For each matching segment:
   - Include the document `id`, `title`, and `source`.
   - Include only the **exact segment text** copied from the document.
   - Do **not rephrase** or **invent** anything — only copy exactly what appears in the document.
   - If multiple documents contributed, return each one with its relevant segment.

2. Do **not return documents** that do not contain any content used in the answer — only include those from which text was actually used.

3. Your output should be a structured list of matching document parts.

Use the following format:

<format_instructions>
{format_instructions}
</format_instructions>

Context documents: <docs>{documents}</docs>
User question: <question>{question}</question>
Generated answer: <answer>{generation}</answer>"""

prompt = PromptTemplate(
    template= system,
    input_variables=["documents", "question", "generation"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Chain
doc_lookup = prompt | llm | parser

# Run
lookup_response = doc_lookup.invoke({"documents":format_docs(docs_to_use), "question": question, "generation": generation})

for id, title, source, segment in zip(lookup_response.id, lookup_response.title, lookup_response.source, lookup_response.segment):
    print(f"ID: {id}\nTitle: {title}\nSource: {source}\nText Segment: {segment}\n")

ID: doc1
Title: Chapter 7
Source: final_draft.pdf
Text Segment: RAG stands for Retrieval-Augmented Generation.

ID: doc1
Title: Chapter 7
Source: final_draft.pdf
Text Segment: RAG is a hybrid approach in which we use pre-trained, parametric-memory generation models along with a non-parametric memory (which acts as an external knowledge source).

ID: doc3
Title: Chapter 7
Source: final_draft.pdf
Text Segment: This technique helps counter the problem of hallucination in LLMs by accessing and utilizing external knowledge.

